In [ ]:
import pandas as pd
import re
import numpy as np
from collections import Counter
from rapidfuzz import fuzz, process

# Load the FORCE CSV file
force_data = pd.read_csv('/home/bipin/Documents/kotak/car_model/manufacturer_data/FORCE.csv')
print(f"Loaded {len(force_data)} records from FORCE.csv")
force_data.head()

# Function to clean and normalize model names
def clean_model(text):
    if pd.isna(text) or text is None:
        return ""
    # Convert to string
    text = str(text).upper()
    # Remove common prefixes and company names
    text = re.sub(r'FORCE\s+MOTORS\s+LIMITED,?\s*A\s+FIRODIA\s+ENTERPRISE,?\s*', '', text)
    text = re.sub(r'FORCE\s+MOTORS\s+LIMITED,?\s*', '', text)
    text = re.sub(r'FORCEMOTOR,?\s*', '', text)
    text = re.sub(r'FORCE\s+', '', text)
    text = re.sub(r'TEMPO\s+', '', text)
    text = re.sub(r'\"', '', text)
    
    # Remove special characters and extra spaces
    text = re.sub(r'[^\w\s\./\-]', ' ', text)  # Keep ., /, and -
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Create a list of cleaned model names
force_data['clean_model'] = force_data['rc_maker_model'].apply(clean_model)

# Filter out too generic or empty model names
generic_terms = ['BS', 'IV', 'III', 'BS-IV', 'BS-III', 'BSIV', 'BSIII', 'BS6', 'BSVI', 'LIMITED']
filtered_models = force_data['clean_model'].apply(
    lambda x: x if x and all(term not in x.split() for term in generic_terms) else np.nan
)

# Count model name frequencies
model_counts = Counter([m for models in filtered_models.dropna() for m in models.split()])
model_counts = {k: v for k, v in model_counts.items() if v >= 3}  # Keep only somewhat frequent terms

# Print top model keywords by frequency
print("\nTop model keywords by frequency:")
sorted_counts = sorted(model_counts.items(), key=lambda x: x[1], reverse=True)
for model, count in sorted_counts[:30]:
    print(f"{model}: {count}")

# Define primary Force model keywords
force_model_keywords = [
    # Main Models
    "TRAVELLER", "TRAX", "TOOFAN", "CRUISER", "GURKHA", "KARGO", "TRUMP",
    
    # Model Variants
    "CHALLENGER", "CLASSIC", "SCHOOL", "MINIBUS", "AMBULANCE", "PICKUP",
    
    # Specific Models and Designations
    "TD 2200", "TD 2650", "TD22", "3700", "3350", "3050", "T1", "4X4", "4X2", "DI",
    "SHAKTI", "MEGA", "MB", "D/VAN", "HARD TOP", "LUXURY",
    
    # Engine/Specification Codes
    "FM2.6", "FM2.6CR", "2370MM", "TR", "HT", "EURO II", "EURO III"
]

# Create alias map for common misspellings and variations
force_alias_map = {
    # Model Aliases
    "TRAVELLAR": "TRAVELLER",
    "TRAVELER": "TRAVELLER", 
    "TRAVELER": "TRAVELLER",
    "TEMPOTRAVELLER": "TRAVELLER",
    "TRAVELLER TD22": "TRAVELLER TD 22",
    "TRAVELLER TD22 BS3": "TRAVELLER TD 22 BS3",
    "TRAVELLER TD22 BSIV": "TRAVELLER TD 22 BS4",
    
    "TRUMO": "TRUMP",
    "TRUMO 40": "TRUMP 40",
    
    "TRAX TOFFAN": "TRAX TOOFAN",
    "TOFFAN": "TOOFAN",
    "TOOFAN IIEG": "TOOFAN",
    
    "CRUISER CLS": "CRUISER CLASSIC",
    "CRUSEIR": "CRUISER",
    
    "KARGO KING": "KARGO",
    
    # Variant Aliases
    "9+D": "9D",
    "13+D": "13D",
    "17+D": "17D",
    "26+D": "26D",
    "11+D": "11D",
    
    "MINI BUS": "MINIBUS",
    "D VAN": "D/VAN",
    
    # Engine/Specification Aliases
    "BSIV": "BS4",
    "BSIII": "BS3",
    "HARD TOP": "HARDTOP",
    "HARD   TOP": "HARDTOP"
}

# Function to normalize text
def normalize(text):
    # Handle NaN/None values or any non-string types
    if pd.isna(text) or text is None:
        return ""
    # Convert to string if it's not already a string
    if not isinstance(text, str):
        text = str(text)
    # Then proceed with normalization
    return re.sub(r'[^A-Z0-9 \.]', '', text.upper())

# Function to extract Force model
def extract_force_model(text):
    txt = normalize(text)
    
    # Skip entries that clearly aren't Force vehicles
    non_force = ["MARUTI", "MAHINDRA", "HONDA", "HYUNDAI", "FORD", "TOYOTA", "TATA", "SKODA", "KIA"]
    for brand in non_force:
        if brand in txt:
            return "NOT FOUND"
    
    # Check for specific model combinations first
    specific_models = [
        "TRAVELLER TD 2200", "TRAVELLER TD 22", "TRAVELLER 3700",
        "TRAX CRUISER CLASSIC", "TRAX CHALLENGER", "TRAX TOOFAN",
        "TOOFAN HARDTOP", "KARGO KING", "TRUMP 40"
    ]
    
    for model in specific_models:
        if model in txt:
            return model
    
    # Fix aliases (correct misspellings and other variations)
    for bad, good in force_alias_map.items():
        if bad in txt:
            txt = txt.replace(bad, good)
    
    # Check for main model families
    if "TRAVELLER" in txt:
        # Look for sub-models
        if "TD 2200" in txt:
            return "TRAVELLER TD 2200"
        if "TD22" in txt or "TD 22" in txt:
            if "SCHOOL" in txt:
                return "TRAVELLER SCHOOL BUS"
            if "MEGA" in txt:
                return "TRAVELLER MEGA BUS"
            return "TRAVELLER TD 22"
        if "3700" in txt:
            return "TRAVELLER 3700"
        if "3350" in txt:
            return "TRAVELLER 3350"
        if "T1" in txt:
            if "D/VAN" in txt:
                return "TRAVELLER T1 D/VAN"
            return "TRAVELLER T1"
        if "LUXURY" in txt:
            return "TRAVELLER LUXURY"
        if "MINIBUS" in txt or "MINI BUS" in txt:
            return "TRAVELLER MINIBUS"
        if "SCHOOL" in txt:
            return "TRAVELLER SCHOOL BUS"
        # If no specific sub-model, return the main model
        return "TRAVELLER"
    
    if "TRAX" in txt:
        if "CRUISER" in txt:
            if "CLASSIC" in txt:
                return "TRAX CRUISER CLASSIC"
            if "AMBULANCE" in txt:
                return "TRAX CRUISER AMBULANCE"
            if "SCHOOL" in txt:
                return "TRAX CRUISER SCHOOL BUS"
            # If just cruiser without specific variant
            return "TRAX CRUISER"
        if "CHALLENGER" in txt:
            return "TRAX CHALLENGER"
        if "TOOFAN" in txt or "TOFFAN" in txt:
            if "HARDTOP" in txt or "HARD TOP" in txt:
                return "TRAX TOOFAN HARDTOP"
            return "TRAX TOOFAN"
        # If just TRAX without specific variant
        return "TRAX"
    
    if "TOOFAN" in txt or "TOFFAN" in txt:
        if "CLASSIC" in txt:
            return "TOOFAN CLASSIC"
        if "HARDTOP" in txt or "HARD TOP" in txt:
            return "TOOFAN HARDTOP"
        if "4X4" in txt:
            return "TOOFAN 4X4"
        # If just TOOFAN without specific variant
        return "TOOFAN"
    
    if "CRUISER" in txt:
        if "CLASSIC" in txt:
            return "CRUISER CLASSIC"
        if "SCHOOL" in txt:
            return "CRUISER SCHOOL BUS"
        if "9D" in txt:
            return "CRUISER 9D"
        if "13D" in txt:
            return "CRUISER 13D"
        # If just CRUISER without specific variant
        return "CRUISER"
    
    if "TRUMP" in txt:
        if "40" in txt or "PICKUP" in txt:
            return "TRUMP 40 PICKUP"
        return "TRUMP"
    
    if "KARGO" in txt or "CARGO" in txt:
        if "KING" in txt:
            return "KARGO KING"
        return "KARGO"
    
    if "GURKHA" in txt:
        if "4X4" in txt:
            return "GURKHA 4X4"
        return "GURKHA"
    
    # Check for generic keywords that might identify the model
    for keyword in sorted(force_model_keywords, key=len, reverse=True):
        if keyword in txt:
            return keyword
    
    # If nothing matches, use fuzzy matching as fallback
    try:
        # Use rapidfuzz for fuzzy matching
        from rapidfuzz import process
        
        # Try to find the best match with a minimum score
        result = process.extractOne(txt, force_model_keywords, scorer=fuzz.token_set_ratio)
        if result and result[1] >= 80:  # if similarity score is at least 80%
            return result[0]
    except Exception:
        # If rapidfuzz isn't available or fails, we'll just continue
        pass
    
    return "NOT FOUND"

# Apply the extraction to the dataset
force_data['force_model'] = force_data['rc_maker_model'].apply(extract_force_model)

# Count the frequency of each extracted model
model_distribution = force_data['force_model'].value_counts()
print("\nExtracted model distribution:")
print(model_distribution.head(20))

# Create final model column with fallback logic
force_data['final_model'] = force_data['force_model']
force_data.loc[force_data['final_model'] == "NOT FOUND", 'final_model'] = "FORCE OTHER"

# Save the processed data
force_data.to_csv('/home/bipin/Documents/kotak/car_model/clean_model/force_processed.csv', index=False)

# Save the model mapping for verification and record
model_mapping = pd.DataFrame({
    'original_model': force_data['rc_maker_model'].unique()
})
model_mapping['cleaned_model'] = model_mapping['original_model'].apply(clean_model)
model_mapping['extracted_model'] = model_mapping['original_model'].apply(extract_force_model)
model_mapping['final_model'] = model_mapping['extracted_model']
model_mapping.loc[model_mapping['final_model'] == "NOT FOUND", 'final_model'] = "FORCE OTHER"

model_mapping.to_csv('/home/bipin/Documents/kotak/car_model/clean_model/force_model_mapping.csv', index=False)

print("\nProcessing complete. Files saved:")
print("1. Processed data: force_processed.csv")
print("2. Model mapping: force_model_mapping.csv")

# Print some statistics
print(f"\nTotal records: {len(force_data)}")
print(f"Unique original models: {len(force_data['rc_maker_model'].unique())}")
print(f"Unique extracted models: {len(force_data['force_model'].unique())}")
print(f"Records mapped to known models: {(force_data['force_model'] != 'NOT FOUND').sum()}")
print(f"Records not mapped (FORCE OTHER): {(force_data['force_model'] == 'NOT FOUND').sum()}")
print(f"Mapping success rate: {(force_data['force_model'] != 'NOT FOUND').sum() / len(force_data) * 100:.2f}%")